First step is to import useful libraries, that will be necessary for the next step of the code.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

We import the dataset

In [ ]:
sms = pd.read_csv('social_media_shares.csv')

We divide data in X and y, where X contains the independent variables, the regressor ones, that help us in predicting the response variable, that is stored in y.

In [ ]:
X = sms.iloc[:,:-1]
y = sms.iloc[:,-1]

We proceed with the Explanatory data analysis (EDA) with visualization, to better understand the relationship between the variables and to understand our dataset better.

We use these function to generate a descriptive statistics that summarize the central tendency, dispersion and shape of our dataset’s distribution.
We want also to see how our dataset is structured in terms of the type of information (numerical,categorical,empty values)

In [ ]:
print(sms.describe())
print(sms.head())
print(sms.info())
print('Social media share dataset shape is', sms.shape)

We want also to see the type of correlation between independent and dependent variable, so we use scatter plot.

In [ ]:
#We plot the relation between each regressor variable with the response variable "shares"
for col in X.columns:
  sns.scatterplot(data=sms,x=col,y='shares')
  plt.show()

We notice that there is not a linear correlation , but actually we can notice something else from these plots.In fact some plots are actually very similar, this is maybe because there is some similiarity are correlation between the regressor variables, so check our assumption we plot the correlation matrix thaks to heat map.

In [ ]:
corr = sms.corr()
plt.figure(figsize = (15,8)) #To set the figure size
sns.heatmap(data=corr, square=True, annot=True, cbar=True)

We can notice that there is a high correlation (almost 0.99) between some variables, meaning that they give a similiary, almost the same information, so we decide to remove and leave only one of the two.
We do this by selecting the upper triangle of the correlation matrix and remove the colum having a correlation value bigger than 0,95.

In [ ]:

upper_tri = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool_))
to_drop = list([column for column in upper_tri.columns if any(upper_tri[column] > 0.95)])
X = X.drop(to_drop, axis=1)
# Let's check the new shape of our X matrix, containing our predictor variables:
# X is now with 56 variables because we removed the useless ones
print('New shape of the X matrix is ',X.shape)

We now proceed in generating a training and test set.

In [ ]:
#Splitting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#Let's see the distribution of our data in the three splits
print("X:train shape: ", X_train.shape)
print("X:test shape: ", X_test.shape)

The next step is feature engineering, so we extract features from raw data, we check for categorical data to manage, remove outliers and so on.
As we saw from the function info we do not have categorical data but we have some variables that are binary (0,1) they look like as one hot coder was already made on them, so we will not scale them.
To avoid scalinge the wrong columns we select their indexes and we proceed with Robust Scaler on all the other columns. 
We used Robust Scalar because it's efficient both for scaling and for the removal of outliers.

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
#We do this in order to avoid scaling columns that have just a binary value (0,1),(as if the One Hot Encoder has done)
#We save the index of the columns to now which columns need to
index_k = X.columns.get_loc('keywords')
index_w = X.columns.get_loc('world')
index_m = X.columns.get_loc('monday')
index_is = X.columns.get_loc('is_weekend')
index_len = len(X.columns)

pipeline = ColumnTransformer([
  ('num', RobustScaler(), list(range(0, index_k + 1))),
  ('num1', RobustScaler(), list(range(index_w + 1, index_m))),
  ('num2', RobustScaler(), list(range(index_is, index_len))),

], remainder='passthrough')
sc = StandardScaler()
X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

We know start implementing the regression models, we want to implement Random Forest Regressor, SVR and Polynomial Regression.

First we want to create a function that gives us the best choices for the hyperparameters.
We proceed with a cross validation function in order to select the best combinations.

In [ ]:
#GRID SEARCH with cross validation
#esplor the parameters to find the best combination
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error

def best_scores_grid_search_df(reg):
    reg.cv_results_
    grid_table = pd.DataFrame(reg.cv_results_)
    colums_wanted = ['params', 'mean_test_score', 'std_test_score', 'rank_test_score']
    grid_table_rank = grid_table[colums_wanted].sort_values(by='rank_test_score', ascending=True)
    return grid_table_rank

To test all different combination of the hyperparameters we create a validation test, that is a small portion of the train test.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.18, random_state=42)
print("X:train shape: ", X_train.shape)
print("X:valid shape: ", X_valid.shape)

We start with SVM, let's find which combination of hyperparameters gives a lower error .

In [ ]:
from sklearn.svm import SVR
#create regressor object
svr =  SVR()
#select the hyperparameters
param_grid_svr = {'kernel': ['poly', 'rbf', 'sigmoid'], 'C': [np.power (10., a) for a in range(-1,3)],'gamma' : ['scale', 'auto']}
reg_svr = GridSearchCV(svr, param_grid_svr, cv=10,scoring='neg_mean_absolute_error')
reg_svr.fit(X_valid,y_valid)
# we print our greed to see the best combination
SVM_best_scores=best_scores_grid_search_df(reg_svr).head()
print(SVM_best_scores)


In [ ]:
#Now that we have found the optimal combination of the parameters we used them for defining the regressor that we will use after.

In [ ]:
# we use these hyperparameters for the regression
svr_best_params = SVM_best_scores.best_params_
svr_regression  = SVR(kernel=svr_best_params['kernel'],C = svr_best_params['C'],gamma= svr_best_params['gamma'])
#our final regressor with the correct parameters is svr_regressor


We know proceed with Random Forest.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# create regressor object
regressor_rf = RandomForestRegressor()
#select the hyperparameters
param_grid_rf ={'max_depth': range(4, 7),'n_estimators': (100, 300, 500)}
rfr_grid = GridSearchCV(regressor_rf,param_grid_rf,cv=10,scoring='neg_mean_absolute_error', verbose=0, n_jobs=-1)
rfr_grid_result = rfr_grid.fit(X_valid, y_valid)
# we print our greed to see the best combination
rfr_best_scores=best_scores_grid_search_df(rfr_grid).head()
print(rfr_best_scores)



Same process for Random Forest

In [ ]:
rfr_best_params = rfr_grid_result.best_params_
rfr = RandomForestRegressor(max_depth=rfr_best_params["max_depth"], n_estimators=rfr_best_params["n_estimators"],random_state=False, verbose=False)
#our final regressor with the correct parameters is rfr

We proceed find the best combination of hyperparameters for Polynomial Regression.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
#Create regressor object
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))
#select the hyperparameters
param_grid_poly = {'polynomialfeatures__degree': np.arange(5), 'linearregression__fit_intercept': [True, False], 'linearregression__normalize': [True, False]}

poly_grid = GridSearchCV(PolynomialRegression(), param_grid_poly, cv=10, scoring='neg_mean_squared_error')

poly_grid_result = poly_grid.fit(X_valid, y_valid)
# we print our greed to see the best combination
poly_best_scores=best_scores_grid_search_df(poly_grid).head()
print(poly_best_scores)


Same process for Polynomial Regression

In [ ]:
#our final regressor with the correct parameters is poly_regressor
poly_best_params = poly_grid_result.best_params_
poly_regressor = PolynomialRegression('polynomialfeatures__degree' = poly_best_params['polynomialfeatures__degree'], 'linearregression__fit_intercept'=poly_best_params['linearregression__fit_intercept'],'linearregression__normalize'=['linearregression__normalize'])


#Now we train our train data with our regression model with the optimal hyperparameters and then we measure it performance on the test set, checking the mean square error, relative error and absolute error.

Starting with th SVM

In [ ]:
svr_regression.fit(X_train,y_train)
y_pred = svr_regression.predict(X_test)
print(mean_squared_error(y_valid,y_pred,squared=False))
print(mean_absolute_error(y_valid, y_pred))
print(mean_absolute_percentage_error(y_valid, y_pred))

We proceed with Random Forest

In [ ]:
rfr.fit(X_train,y_train)
y_pred = rfr.predict(X_test)
print(mean_squared_error(y_valid,y_pred,squared=False))
print(mean_absolute_error(y_valid, y_pred))
print(mean_absolute_percentage_error(y_valid, y_pred))

Lastly Polynomial

In [ ]:
poly_regressor.fit(X_train,y_train)
y_pred = poly_regressor.predict(X_test)
print(mean_squared_error(y_valid,y_pred,squared=False))
print(mean_absolute_error(y_valid, y_pred))
print(mean_absolute_percentage_error(y_valid, y_pred))

For the final task of the project we select a subset of the attributes(only the most relevant) and with it we train and test again our best model.
Since our best model is SVM we will try the subset of attributes only on it.

From our analysis the most relevant columns are:

● tokens_title: Number of words in the title 0
● tokens_content: Number of words in the content 1
● hrefs: Number of links 5
● imgs: Number of images 7
● videos: Number of videos 8
● lifestyle: Topic Lifestyle 11
● entertainment: Topic Entertainment 12
● bus: Topic Business 13
● socmed: Topic Social Media 14
● tech: Topic Tech 15
● world: Topic World 16
● self_reference_min_shares: Min. shares of referenced articles in Mashable 26
● self_reference_max_shares: Max. shares of referenced articles in Mashable 27
● self_reference_avg_shares: Avg. shares of referenced articles in Mashable 28
● global_sentiment_polarity: Text sentiment polarity 43
● global_rate_positive_words: Rate of positive words in the content 44
● global_rate_negative_words: Rate of negative words in the content 45
● title_sentiment_polarity: Title polarity 55
● abs_title_subjectivity: Absolute subjectivity level 56
● abs_title_sentiment_polarity: Absolute polarity level 57

We create a new X test only with this regressor variables

In [ ]:
#We selected the indexes we are interested in
index1 = X.columns.get_loc('tokens_title')
index2 = X.columns.get_loc('tokens_content')
index3 = X.columns.get_loc('hrefs')
index4 = X.columns.get_loc('imgs')
index5 = X.columns.get_loc('videos')
index6 = X.columns.get_loc('lifestyle')
index7 = X.columns.get_loc('entertainment')
index8 = X.columns.get_loc('bus')
index9 = X.columns.get_loc('socmed')
index10 = X.columns.get_loc('tech')
index11 = X.columns.get_loc('world')
index12 = X.columns.get_loc('self_reference_min_shares')
index13 = X.columns.get_loc('self_reference_max_shares')
index14 = X.columns.get_loc('self_reference_avg_shares')
index15 = X.columns.get_loc('sentiment_polarity')
index16 = X.columns.get_loc('global_rate_positive_words')
index17 = X.columns.get_loc('global_rate_negative_words')
index18 = X.columns.get_loc('title_sentiment_polarity')
index19 = X.columns.get_loc('abs_title_subjectivity')
index20 = X.columns.get_loc('abs_title_sentiment_polarity')
index_len2 = len(X.columns)
#We create a list with the indexes
index_lst = [index1, index2, index3, index4, index5, index6, index7, index8, index9, index10, index11, index12, index13, index14, index15, index16, index17, index18, index19, index20]
#We make them the columns of our X
new_X= X.iloc[:, index_lst]
print(new_X)


We split in train and test the new X

In [ ]:
new_X_train,new_X_test,y_train,y_test= train_test_split(new_X,y,train_size=0.78,random_state=42)
print("new_X train shape: ",new_X_train.shape)
print("new_X test shape: ",new_X_test.shape)

We scale the new X

In [ ]:
index_len2 = len(new_X.columns)
pipeline = ColumnTransformer([
    ('num',RobustScaler(),list(range(0,index5+1))),
    ('num1',RobustScaler(),list(range(index12, index_len2))),
],remainder='passthrough')
new_X_train = pipeline.fit_transform(new_X_train)
new_X_test = pipeline.transform(new_X_test)

We can final try our best model on the  new X train:

In [ ]:
svr_regression.fit(new_X_train,y_train)
y_pred = svr_regression.predict(new_X_test)
print(mean_squared_error(y_valid,y_pred,squared=False))
print(mean_absolute_error(y_valid, y_pred))
print(mean_absolute_percentage_error(y_valid, y_pred))